# STS Benchmark Datasets

### Preparation

Setup all required libraries

In [1]:
import logging
import re
import sys

import numpy as np
import pandas as pd

from gensim.models.keyedvectors import KeyedVectors, FastTextKeyedVectors

from fse.models import Average, SIF, uSIF
from fse import CSplitIndexedList

from re import sub

from scipy.stats import pearsonr

from nltk import word_tokenize

logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

Next, we require the sentences from the STS benchmark dataset.

In [2]:
file= "../evaluation/sts-test.csv"
similarities, sent_a, sent_b = [], [], []
with open(file, "r") as f:
    for l in f:
        line = l.rstrip().split("\t")
        similarities.append(float(line[4]))
        sent_a.append(line[5])
        sent_b.append(line[6])
similarities = np.array(similarities)
assert len(similarities) == len(sent_a) == len(sent_b)
task_length = len(similarities)

for i, obj in enumerate(zip(similarities, sent_a, sent_b)):
    print(f"{i}\tSim: {obj[0].round(3):.1f}\t{obj[1]:40s}\t{obj[2]:40s}\t")
    if i == 4:
        break

0	Sim: 2.5	A girl is styling her hair.             	A girl is brushing her hair.            	
1	Sim: 3.6	A group of men play soccer on the beach.	A group of boys are playing soccer on the beach.	
2	Sim: 5.0	One woman is measuring another woman's ankle.	A woman measures another woman's ankle. 	
3	Sim: 4.2	A man is cutting up a cucumber.         	A man is slicing a cucumber.            	
4	Sim: 1.5	A man is playing a harp.                	A man is playing a keyboard.            	


Each of these sentence requires some preparation (i.e. tokenization) to be used in the core input formats.
To reproduce the results from the uSIF paper this part is taken from https://github.com/kawine/usif/blob/master/usif.py

In [3]:
not_punc = re.compile('.*[A-Za-z0-9].*')

def prep_token(token):
    t = token.lower().strip("';.:()").strip('"')
    t = 'not' if t == "n't" else t
    return re.split(r'[-]', t)

def prep_sentence(sentence):
    tokens = []
    for token in word_tokenize(sentence):
        if not_punc.match(token):
            tokens = tokens + prep_token(token)
    return tokens

Next we define the IndexedList object. The IndexedList extends the previously constructed sent_a and sent_b list together. We additionally provide a custom function "prep_sentence" which performs all the prepocessing for a single sentence. Therefore we need the extention **CSplitIndexedList**, which provides you the option to provide a custom split function

In [4]:
sentences = CSplitIndexedList(sent_a, sent_b, custom_split=prep_sentence)

The IndexedList returns the core object required for fse to train a sentence embedding: A tuple. This object constists of words (a list of strings) and its corresponding index. The latter is important if multiple cores access the input queue simultaneously. Thus it must be always provided. The index represents the row in the matrix where it can later be found.

In [5]:
sentences[0]

(['a', 'girl', 'is', 'styling', 'her', 'hair'], 0)

Note, that IndexedList does not convert the sentences inplace but only on calling the __getitem__ method in order to turn the sentence into a tuple. You can access the original sentence using

In [6]:
sentences.items[0]

'A girl is styling her hair.'

### Loading the models

It is required for us to load the models as BaseKeyedVectors or as an BaseWordEmbeddingsModel. For this notebook, I already converted the models to a BaseKeyedVectors instance and saved the corresponding instance on my external harddrive. You have to replicate these steps yourself, because getting all the files can be a bit difficult, as the total filesize is around 15 GB.

In [7]:
models, results = {}, {}

The following code performs a disk-to-ram training. Passing a path to __wv_mapfile_path__ will store the corresponding word vectors (wv) as a numpy memmap array. This is required, because loading all vectors into ram would would take up a lot of storage unecessary. The wv.vectors file will be replace by its memmap representation, which is why the next models do not require the wv_mapfile_path argument, as they access the same memmap object.

The lang_freq="en" induces the frequencies of the words according to the wordfreq package. This functionality allows you to work with pre-trained embeddings which don't come with frequency information. The method overwrites the counts in the glove.wv.vocab class, so that all further models also benefit from this induction.

In [ ]:
glove = KeyedVectors.load(path_to_models+"glove.840B.300d.model")

print(f"Before memmap {sys.getsizeof(glove.vectors)}")

models[f"CBOW-Glove"] = Average(glove, wv_mapfile_path="data/glove", lang_freq="en")

print(f"After memmap {sys.getsizeof(glove.vectors)}")

models[f"SIF-Glove"] = SIF(glove, components=15)
models[f"uSIF-Glove"] = uSIF(glove,length=11)

In [ ]:
# Do all the vectors contain the same content?
(models[f"SIF-Glove"].wv.vectors == models[f"uSIF-Glove"].wv.vectors).all()

Another option is to load the KeyedVectors using the kwarg mmap="r"

In [ ]:
w2v = KeyedVectors.load(path_to_models+"google_news.model", mmap="r")

models[f"CBOW-W2V"] = Average(w2v, lang_freq="en")
models[f"SIF-W2V"] = SIF(w2v, components=10)
models[f"uSIF-W2V"] = uSIF(w2v, length=11)

In [ ]:
ft = FastTextKeyedVectors.load(path_to_models+"ft_crawl_300d_2m.model", mmap="r")
models[f"CBOW-FT"] = Average(ft, lang_freq="en")
models[f"SIF-FT"] = SIF(ft, components=10)
models[f"uSIF-FT"] = uSIF(ft, length=11)

In [ ]:
paranmt = KeyedVectors.load(path_to_models+"paranmt.model", mmap="r")

models[f"CBOW-Paranmt"] = Average(paranmt, lang_freq="en")
models[f"SIF-Paranmt"] = SIF(paranmt, components=10)
models[f"uSIF-Paranmt"] = uSIF(paranmt, length=11)

In [ ]:
paragram = KeyedVectors.load(path_to_models+"paragram_sl999_czeng.model", mmap="r")

models[f"CBOW-Paragram"] = Average(paragram, lang_freq="en")
models[f"SIF-Paragram"] = SIF(paragram, components=10)
models[f"uSIF-Paragram"] = uSIF(paragram, length=11)

## Computation of the results for the STS benchmark

We are finally able to compute the STS benchmark values for all models.

In [ ]:
# This function is used to compute the similarities between two sentences.
# Task length is the length of the sts dataset.
def compute_similarities(task_length, model):
    sims = []
    for i, j in zip(range(task_length), range(task_length, 2*task_length)):
        sims.append(model.sv.similarity(i,j))
    return sims

In [ ]:
for k, m in models.items():
    m_type  = k.split("-")[0]
    emb_type = k.split("-")[1]
    m.train(sentences)
    r = pearsonr(similarities, compute_similarities(task_length, m))[0].round(4) * 100
    results[f"{m_type}-{emb_type}"] = r
    print(k, f"{r:2.2f}")

In [ ]:
pd.DataFrame.from_dict(results, orient="index", columns=["Pearson"])

If you closely study the values above you will find that:
- SIF-Glove is almost equivalent to the values reported in http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark
- CBOW-Paranmt is a little better than ParaNMT Word Avg. in https://www.aclweb.org/anthology/W18-3012
- uSIF-Paranmt is a little worse than ParaNMT+UP in https://www.aclweb.org/anthology/W18-3012
- uSIF-Paragram is a little worse than PSL+UP in https://www.aclweb.org/anthology/W18-3012

However, I guess those differences might arise due to differences in preprocessing. Too bad we didn't hit 80. If you have any ideas why those values don't match exactly, feel free to contact me anytime.